# AFRIMASH CUSTOMER INTELLIGENCE CHALLENGE
## Complete End-to-End Analysis Pipeline

**Objective:** Build a comprehensive customer intelligence system that:
1. Segments customers into meaningful groups
2. Predicts customer behavior and purchasing patterns
3. Analyzes and improves customer retention rates
4. Generates personalized product recommendations

---

## 📦 PART 1: Setup & Import Libraries

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    mean_squared_error, r2_score, mean_absolute_error, accuracy_score, silhouette_score
)

# For recommendations
from collections import defaultdict
from itertools import combinations
import pickle

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)
%matplotlib inline

print("✅ All libraries imported successfully!")

## 📁 PART 2: Load Data

In [ ]:
# Load the datasets
print("Loading datasets...")

try:
    rfm_df = pd.read_excel('Copy of RFM_Data.xlsx')
    trans_df = pd.read_excel('Copy of Transaction_Data.xlsx')
    
    print(f"✅ RFM Data loaded: {rfm_df.shape[0]:,} customers, {rfm_df.shape[1]} columns")
    print(f"✅ Transaction Data loaded: {trans_df.shape[0]:,} transactions, {trans_df.shape[1]} columns")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Please ensure the Excel files are in the same directory as this notebook.")

## 🔍 PART 3: Initial Data Exploration

In [ ]:
# RFM Data Overview
print("="*80)
print("RFM DATA OVERVIEW")
print("="*80)
print("\nFirst 5 rows:")
display(rfm_df.head())

print("\nData Info:")
print(rfm_df.info())

print("\nBasic Statistics:")
display(rfm_df.describe())

print("\nMissing Values:")
missing_rfm = rfm_df.isnull().sum()
print(missing_rfm[missing_rfm > 0])

In [ ]:
# Transaction Data Overview
print("="*80)
print("TRANSACTION DATA OVERVIEW")
print("="*80)
print("\nFirst 5 rows:")
display(trans_df.head())

print("\nData Info:")
print(trans_df.info())

print("\nBasic Statistics:")
display(trans_df.describe())

print("\nMissing Values:")
missing_trans = trans_df.isnull().sum()
print(missing_trans[missing_trans > 0])

## 🧹 PART 4: Data Cleaning

In [ ]:
print("="*80)
print("DATA CLEANING")
print("="*80)

# 4.1 Handle missing products
if 'Product(s)' in trans_df.columns:
    missing_products = trans_df['Product(s)'].isnull().sum()
    trans_df['Product(s)'].fillna("Unknown Product", inplace=True)
    print(f"✓ Filled {missing_products:,} missing product names with 'Unknown Product'")

# 4.2 Check for negative values
if 'Revenue' in trans_df.columns:
    negative_revenue = trans_df[trans_df['Revenue'] < 0]
    print(f"✓ Found {len(negative_revenue):,} negative revenue transactions (refunds/returns)")

if 'Monetary' in rfm_df.columns:
    negative_monetary = rfm_df[rfm_df['Monetary'] < 0]
    print(f"✓ Found {len(negative_monetary):,} customers with negative monetary values")

# 4.3 Remove duplicates
if 'Order #' in trans_df.columns:
    initial_trans = len(trans_df)
    trans_df = trans_df.drop_duplicates(subset=['Order #'], keep='first')
    print(f"✓ Removed {initial_trans - len(trans_df):,} duplicate orders")

# 4.4 Convert date columns
if 'Date' in trans_df.columns:
    trans_df['Date'] = pd.to_datetime(trans_df['Date'])
    print(f"✓ Converted Date column to datetime")

print("\n✅ Data cleaning complete!")

## ⚙️ PART 5: Feature Engineering

In [ ]:
print("="*80)
print("FEATURE ENGINEERING")
print("="*80)

# 5.1 Calculate Recency
current_date = trans_df['Date'].max()
print(f"\nAnalysis Date: {current_date}")

last_purchase = trans_df.groupby('Customer_ID')['Date'].max().reset_index()
last_purchase.columns = ['Customer_ID', 'Last_Purchase_Date']
last_purchase['Recency'] = (current_date - last_purchase['Last_Purchase_Date']).dt.days

rfm_df = rfm_df.merge(last_purchase[['Customer_ID', 'Recency', 'Last_Purchase_Date']], 
                       on='Customer_ID', how='left')
print(f"✓ Added Recency (days since last purchase)")

# 5.2 Extract time features
trans_df['Year'] = trans_df['Date'].dt.year
trans_df['Month'] = trans_df['Date'].dt.month
trans_df['Quarter'] = trans_df['Date'].dt.quarter
trans_df['DayOfWeek'] = trans_df['Date'].dt.dayofweek
trans_df['DayName'] = trans_df['Date'].dt.day_name()
trans_df['MonthName'] = trans_df['Date'].dt.month_name()
print("✓ Added time-based features")

# 5.3 First purchase date
first_purchase = trans_df.groupby('Customer_ID')['Date'].min().reset_index()
first_purchase.columns = ['Customer_ID', 'First_Purchase_Date']
rfm_df = rfm_df.merge(first_purchase, on='Customer_ID', how='left')
print("✓ Added First Purchase Date")

# 5.4 Customer Age
rfm_df['Customer_Age_Days'] = (current_date - rfm_df['First_Purchase_Date']).dt.days
print("✓ Added Customer Age")

# 5.5 Frequency Categories
def categorize_frequency(freq):
    if freq == 1:
        return 'One-time'
    elif freq <= 3:
        return 'Low'
    elif freq <= 10:
        return 'Medium'
    elif freq <= 20:
        return 'High'
    else:
        return 'Very High'

rfm_df['Frequency_Category'] = rfm_df['Frequency'].apply(categorize_frequency)
print("✓ Added Frequency Category")

# 5.6 Monetary Categories
def categorize_monetary(value):
    if value < 50000:
        return 'Low Value'
    elif value < 200000:
        return 'Medium Value'
    elif value < 1000000:
        return 'High Value'
    else:
        return 'Very High Value'

rfm_df['Monetary_Category'] = rfm_df['Monetary'].apply(categorize_monetary)
print("✓ Added Monetary Category")

# 5.7 Recency Categories
def categorize_recency(recency):
    if recency <= 30:
        return 'Active (0-30 days)'
    elif recency <= 90:
        return 'Recent (31-90 days)'
    elif recency <= 180:
        return 'Cooling (91-180 days)'
    elif recency <= 365:
        return 'At Risk (181-365 days)'
    else:
        return 'Lost (>365 days)'

rfm_df['Recency_Category'] = rfm_df['Recency'].apply(categorize_recency)
print("✓ Added Recency Category")

# 5.8 Product Categories
def extract_product_category(product_text):
    if pd.isna(product_text) or product_text == "Unknown Product":
        return "Unknown"
    
    product_lower = str(product_text).lower()
    
    if any(word in product_lower for word in ['chick', 'pullet', 'broiler', 'poultry', 'bird']):
        return 'Poultry'
    elif any(word in product_lower for word in ['seed', 'maize', 'corn', 'soybean', 'rice']):
        return 'Seeds'
    elif any(word in product_lower for word in ['feed', 'concentrate', 'premix']):
        return 'Feed'
    elif any(word in product_lower for word in ['fertilizer', 'npk', 'urea']):
        return 'Fertilizer'
    elif any(word in product_lower for word in ['pesticide', 'herbicide', 'insecticide', 'fungicide']):
        return 'Agrochemicals'
    elif any(word in product_lower for word in ['tractor', 'machine', 'equipment', 'processing', 'pump']):
        return 'Equipment'
    elif any(word in product_lower for word in ['vegetable', 'tomato', 'pepper', 'cucumber', 'carrot']):
        return 'Vegetables'
    elif any(word in product_lower for word in ['fruit', 'papaya', 'mango', 'watermelon']):
        return 'Fruits'
    else:
        return 'Other'

trans_df['Product_Category'] = trans_df['Product(s)'].apply(extract_product_category)
print("✓ Extracted Product Categories")

# 5.9 Category counts per customer
category_counts = trans_df.groupby('Customer_ID')['Product_Category'].value_counts().unstack(fill_value=0)
category_counts.columns = [f'Category_{col}' for col in category_counts.columns]
rfm_df = rfm_df.merge(category_counts, on='Customer_ID', how='left')
rfm_df = rfm_df.fillna(0)
print("✓ Added product category purchase counts")

# 5.10 Days between purchases
rfm_df['Days_Between_Purchases'] = rfm_df['Customer_Age_Days'] / rfm_df['Frequency'].replace(0, 1)
print("✓ Added average days between purchases")

print("\n✅ Feature engineering complete!")
print(f"\nTotal features in RFM dataset: {rfm_df.shape[1]}")

## 📊 PART 6: Exploratory Data Analysis (EDA)

### 6.1 Customer Overview Statistics

In [ ]:
print("="*80)
print("CUSTOMER OVERVIEW STATISTICS")
print("="*80)

print("\n--- Customer Type Distribution ---")
print(rfm_df['Customer_Type'].value_counts())
print(f"\nNew Customers: {(rfm_df['Customer_Type']=='new').sum()/len(rfm_df)*100:.1f}%")
print(f"Returning Customers: {(rfm_df['Customer_Type']=='returning').sum()/len(rfm_df)*100:.1f}%")

print("\n--- Frequency Category Distribution ---")
print(rfm_df['Frequency_Category'].value_counts())

print("\n--- Monetary Category Distribution ---")
print(rfm_df['Monetary_Category'].value_counts())

print("\n--- Recency Category Distribution ---")
print(rfm_df['Recency_Category'].value_counts())

### 6.2 Product Analysis

In [ ]:
print("--- Top Product Categories ---")
print(trans_df['Product_Category'].value_counts())

print("\n--- Revenue by Product Category ---")
category_revenue = trans_df.groupby('Product_Category')['Revenue'].sum().sort_values(ascending=False)
print(category_revenue)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Category counts
trans_df['Product_Category'].value_counts().head(10).plot(kind='barh', ax=axes[0], color='teal')
axes[0].set_title('Top 10 Product Categories by Transaction Count', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Number of Transactions')

# Category revenue
category_revenue.head(10).plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Top 10 Product Categories by Revenue', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Total Revenue (₦)')

plt.tight_layout()
plt.show()

### 6.3 Churn Analysis

In [ ]:
print("--- Churn Analysis ---")
churned_customers = rfm_df[rfm_df['Recency'] > 90]
print(f"Total Customers: {len(rfm_df):,}")
print(f"At Risk/Churned (>90 days): {len(churned_customers):,} ({len(churned_customers)/len(rfm_df)*100:.1f}%)")
print(f"Revenue from Churned Customers: ₦{churned_customers['Monetary'].sum():,.2f}")
print(f"Active Customers (<=30 days): {len(rfm_df[rfm_df['Recency'] <= 30]):,} ({len(rfm_df[rfm_df['Recency'] <= 30])/len(rfm_df)*100:.1f}%)")

### 6.4 Top Customers

In [ ]:
print("--- Top 10 Customers by Revenue ---")
top_customers = rfm_df.nlargest(10, 'Monetary')[['Customer_ID', 'Monetary', 'Frequency', 'Recency']]
display(top_customers)

### 6.5 Comprehensive Visualizations

In [ ]:
# Create comprehensive EDA dashboard
fig, axes = plt.subplots(3, 3, figsize=(20, 15))
fig.suptitle('AFRIMASH CUSTOMER INTELLIGENCE - EXPLORATORY DATA ANALYSIS', fontsize=20, fontweight='bold')

# 1. Frequency Distribution
axes[0, 0].hist(rfm_df['Frequency'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Frequency Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Number of Purchases')
axes[0, 0].set_ylabel('Number of Customers')
axes[0, 0].axvline(rfm_df['Frequency'].median(), color='red', linestyle='--', 
                   label=f'Median: {rfm_df["Frequency"].median():.0f}')
axes[0, 0].legend()

# 2. Monetary Distribution (log scale)
axes[0, 1].hist(np.log10(rfm_df['Monetary'][rfm_df['Monetary'] > 0]), bins=50, 
                color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Monetary Value Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Log10(Total Spent)')
axes[0, 1].set_ylabel('Number of Customers')

# 3. Recency Distribution
axes[0, 2].hist(rfm_df['Recency'], bins=50, color='lightcoral', edgecolor='black')
axes[0, 2].set_title('Recency Distribution', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Days Since Last Purchase')
axes[0, 2].set_ylabel('Number of Customers')
axes[0, 2].axvline(90, color='red', linestyle='--', label='90 days (Churn threshold)')
axes[0, 2].legend()

# 4. Customer Type
customer_type_counts = rfm_df['Customer_Type'].value_counts()
axes[1, 0].pie(customer_type_counts, labels=customer_type_counts.index, autopct='%1.1f%%', 
               colors=['#ff9999', '#66b3ff'], startangle=90)
axes[1, 0].set_title('Customer Type Distribution', fontsize=14, fontweight='bold')

# 5. Frequency Category
freq_cat = rfm_df['Frequency_Category'].value_counts()
axes[1, 1].bar(freq_cat.index, freq_cat.values, color='purple', alpha=0.7)
axes[1, 1].set_title('Frequency Categories', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Category')
axes[1, 1].set_ylabel('Number of Customers')
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Recency Category
rec_cat = rfm_df['Recency_Category'].value_counts()
axes[1, 2].barh(rec_cat.index, rec_cat.values, color='orange', alpha=0.7)
axes[1, 2].set_title('Recency Categories', fontsize=14, fontweight='bold')
axes[1, 2].set_xlabel('Number of Customers')

# 7. Product Categories
prod_cat = trans_df['Product_Category'].value_counts().head(8)
axes[2, 0].bar(prod_cat.index, prod_cat.values, color='teal', alpha=0.7)
axes[2, 0].set_title('Top Product Categories', fontsize=14, fontweight='bold')
axes[2, 0].set_xlabel('Category')
axes[2, 0].set_ylabel('Number of Transactions')
axes[2, 0].tick_params(axis='x', rotation=45)

# 8. Monthly Revenue Trend
monthly_revenue = trans_df.groupby(['Year', 'Month'])['Revenue'].sum().reset_index()
monthly_revenue['YearMonth'] = monthly_revenue['Year'].astype(str) + '-' + monthly_revenue['Month'].astype(str).str.zfill(2)
recent_months = monthly_revenue.tail(24)
axes[2, 1].plot(range(len(recent_months)), recent_months['Revenue'], marker='o', color='green', linewidth=2)
axes[2, 1].set_title('Monthly Revenue Trend (Last 24 Months)', fontsize=14, fontweight='bold')
axes[2, 1].set_xlabel('Month')
axes[2, 1].set_ylabel('Revenue (₦)')
axes[2, 1].tick_params(axis='x', rotation=45)
axes[2, 1].grid(True, alpha=0.3)

# 9. Revenue by Customer Type
customer_type_revenue = rfm_df.groupby('Customer_Type')['Monetary'].sum()
axes[2, 2].bar(customer_type_revenue.index, customer_type_revenue.values, color=['#ff9999', '#66b3ff'])
axes[2, 2].set_title('Total Revenue by Customer Type', fontsize=14, fontweight='bold')
axes[2, 2].set_xlabel('Customer Type')
axes[2, 2].set_ylabel('Total Revenue (₦)')

plt.tight_layout()
plt.savefig('eda_dashboard.png', dpi=300, bbox_inches='tight')
print("✓ Saved: eda_dashboard.png")
plt.show()

In [ ]:
# Additional Deep Dive Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('AFRIMASH DEEP DIVE ANALYSIS', fontsize=18, fontweight='bold')

# 1. RFM Scatter Plot
scatter = axes[0, 0].scatter(rfm_df['Recency'], rfm_df['Frequency'], 
                            c=rfm_df['Monetary'], cmap='viridis', 
                            alpha=0.6, s=50)
axes[0, 0].set_title('RFM Analysis: Recency vs Frequency', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Recency (Days)')
axes[0, 0].set_ylabel('Frequency (Purchases)')
axes[0, 0].axvline(90, color='red', linestyle='--', alpha=0.5, label='90-day threshold')
axes[0, 0].legend()
plt.colorbar(scatter, ax=axes[0, 0], label='Monetary Value (₦)')

# 2. Revenue by Category
cat_rev = trans_df.groupby('Product_Category')['Revenue'].sum().sort_values(ascending=True).tail(10)
axes[0, 1].barh(cat_rev.index, cat_rev.values, color='coral')
axes[0, 1].set_title('Revenue by Product Category (Top 10)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Total Revenue (₦)')

# 3. Customer Lifetime Distribution
axes[1, 0].hist(rfm_df['Customer_Age_Days'], bins=50, color='mediumpurple', edgecolor='black')
axes[1, 0].set_title('Customer Lifetime Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Days Since First Purchase')
axes[1, 0].set_ylabel('Number of Customers')
axes[1, 0].axvline(rfm_df['Customer_Age_Days'].median(), color='red', 
                   linestyle='--', label=f'Median: {rfm_df["Customer_Age_Days"].median():.0f} days')
axes[1, 0].legend()

# 4. Day of Week Analysis
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_trans = trans_df['DayName'].value_counts().reindex(day_order)
axes[1, 1].bar(day_trans.index, day_trans.values, color='steelblue', alpha=0.7)
axes[1, 1].set_title('Transactions by Day of Week', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Number of Transactions')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('deep_dive_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: deep_dive_analysis.png")
plt.show()

## 💾 PART 7: Save Cleaned Data

In [ ]:
# Save cleaned datasets
rfm_df.to_csv('rfm_clean.csv', index=False)
trans_df.to_csv('transactions_clean.csv', index=False)

print("✓ Saved: rfm_clean.csv")
print("✓ Saved: transactions_clean.csv")

# Save summary statistics
summary_stats = {
    'Metric': [
        'Total Customers',
        'Total Transactions',
        'Total Revenue',
        'Average Customer Value',
        'Average Transaction Value',
        'Median Frequency',
        'Median Recency (days)',
        'Active Customers (0-30 days)',
        'At Risk Customers (>90 days)',
        'One-time Buyers',
        'Returning Customers Rate',
        'Average Purchase Frequency',
        'Most Popular Category'
    ],
    'Value': [
        f"{len(rfm_df):,}",
        f"{len(trans_df):,}",
        f"₦{trans_df['Revenue'].sum():,.2f}",
        f"₦{rfm_df['Monetary'].mean():,.2f}",
        f"₦{trans_df['Revenue'].mean():,.2f}",
        f"{rfm_df['Frequency'].median():.0f}",
        f"{rfm_df['Recency'].median():.0f}",
        f"{len(rfm_df[rfm_df['Recency'] <= 30]):,} ({len(rfm_df[rfm_df['Recency'] <= 30])/len(rfm_df)*100:.1f}%)",
        f"{len(rfm_df[rfm_df['Recency'] > 90]):,} ({len(rfm_df[rfm_df['Recency'] > 90])/len(rfm_df)*100:.1f}%)",
        f"{len(rfm_df[rfm_df['Frequency'] == 1]):,} ({len(rfm_df[rfm_df['Frequency'] == 1])/len(rfm_df)*100:.1f}%)",
        f"{(rfm_df['Customer_Type']=='returning').sum()/len(rfm_df)*100:.1f}%",
        f"{rfm_df['Frequency'].mean():.2f}",
        trans_df['Product_Category'].mode()[0]
    ]
}

summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv('summary_statistics.csv', index=False)
print("✓ Saved: summary_statistics.csv")

print("\n📊 KEY INSIGHTS SUMMARY:")
print("="*80)
display(summary_df)

## 👥 PART 8: Customer Segmentation

### 8.1 RFM Segmentation

In [ ]:
print("="*80)
print("RFM SEGMENTATION")
print("="*80)

# Calculate RFM Scores (1-5 scale)
rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], q=5, labels=[5,4,3,2,1], duplicates='drop')
rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), q=5, labels=[1,2,3,4,5], duplicates='drop')
rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], q=5, labels=[1,2,3,4,5], duplicates='drop')

# Convert to numeric
rfm_df['R_Score'] = rfm_df['R_Score'].astype(int)
rfm_df['F_Score'] = rfm_df['F_Score'].astype(int)
rfm_df['M_Score'] = rfm_df['M_Score'].astype(int)

# Calculate overall RFM Score
rfm_df['RFM_Score'] = rfm_df['R_Score'] + rfm_df['F_Score'] + rfm_df['M_Score']

print(f"✓ Calculated RFM Scores (Range: {rfm_df['RFM_Score'].min()}-{rfm_df['RFM_Score'].max()})")

# Define RFM Segments
def get_rfm_segment(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 4:
        return 'Loyal Customers'
    elif m >= 4 and f <= 3:
        return 'Big Spenders'
    elif r >= 4 and f <= 2:
        return 'Promising'
    elif r >= 3 and f >= 2 and m >= 2:
        return 'Needs Attention'
    elif r == 2 or r == 3:
        return 'About to Sleep'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    elif f >= 4 and m >= 4:
        return "Can't Lose Them"
    elif r <= 2 and f <= 2 and m >= 2:
        return 'Hibernating'
    else:
        return 'Lost'

rfm_df['RFM_Segment'] = rfm_df.apply(get_rfm_segment, axis=1)

print(f"✓ Created {rfm_df['RFM_Segment'].nunique()} RFM segments\n")
print("Segment Distribution:")
print(rfm_df['RFM_Segment'].value_counts())

### 8.2 K-Means Clustering

In [ ]:
print("\n" + "="*80)
print("K-MEANS CLUSTERING")
print("="*80)

# Select features for clustering
clustering_features = ['Recency', 'Frequency', 'Monetary', 'Avg_Order_Value',
                       'Customer_Age_Days', 'Purchase_Rate']

X = rfm_df[clustering_features].fillna(0)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means
optimal_k = 5
print(f"\nUsing {optimal_k} clusters...")

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
rfm_df['Cluster'] = kmeans.fit_predict(X_scaled)

# Name clusters
def name_cluster(cluster_id, cluster_data):
    avg_recency = cluster_data['Recency'].mean()
    avg_frequency = cluster_data['Frequency'].mean()
    avg_monetary = cluster_data['Monetary'].mean()
    
    if avg_monetary > rfm_df['Monetary'].quantile(0.75) and avg_frequency > 10 and avg_recency < 180:
        return 'Loyal High-Value'
    elif avg_monetary > rfm_df['Monetary'].quantile(0.5) and avg_recency > 365:
        return 'Dormant Veterans'
    elif avg_monetary < rfm_df['Monetary'].quantile(0.25) and avg_recency > 365:
        return 'Lost Low-Value'
    elif avg_recency > 180 and avg_frequency < 10:
        return 'Mid-Tier Customers'
    elif avg_recency < 90:
        return 'Recent Engagers'
    return f'Cluster_{cluster_id}'

cluster_names = {}
for cluster_id in range(optimal_k):
    cluster_data = rfm_df[rfm_df['Cluster'] == cluster_id]
    cluster_names[cluster_id] = name_cluster(cluster_id, cluster_data)

rfm_df['Cluster_Name'] = rfm_df['Cluster'].map(cluster_names)

print(f"✓ Named {optimal_k} clusters\n")
print("Cluster Distribution:")
print(rfm_df['Cluster_Name'].value_counts())

### 8.3 Segment Analysis

In [ ]:
# RFM Segment Summary
rfm_segment_summary = rfm_df.groupby('RFM_Segment').agg({
    'Customer_ID': 'count',
    'Monetary': ['sum', 'mean'],
    'Frequency': 'mean',
    'Recency': 'mean',
    'Avg_Order_Value': 'mean'
}).round(2)

rfm_segment_summary.columns = ['_'.join(col).strip('_') for col in rfm_segment_summary.columns]
rfm_segment_summary = rfm_segment_summary.reset_index()
rfm_segment_summary = rfm_segment_summary.sort_values('Monetary_sum', ascending=False)

print("\n📊 RFM Segment Summary:")
display(rfm_segment_summary[['RFM_Segment', 'Customer_ID_count', 'Monetary_sum', 'Monetary_mean']])

# Cluster Summary
cluster_summary = rfm_df.groupby('Cluster_Name').agg({
    'Customer_ID': 'count',
    'Monetary': ['sum', 'mean'],
    'Frequency': 'mean',
    'Recency': 'mean',
    'Avg_Order_Value': 'mean'
}).round(2)

cluster_summary.columns = ['_'.join(col).strip('_') for col in cluster_summary.columns]
cluster_summary = cluster_summary.reset_index()
cluster_summary = cluster_summary.sort_values('Monetary_sum', ascending=False)

print("\n🤖 K-Means Cluster Summary:")
display(cluster_summary[['Cluster_Name', 'Customer_ID_count', 'Monetary_sum', 'Monetary_mean']])

### 8.4 Save Segmentation Results

In [ ]:
# Save segmented data
rfm_df.to_csv('rfm_segmented.csv', index=False)
rfm_segment_summary.to_csv('rfm_segment_summary.csv', index=False)
cluster_summary.to_csv('cluster_summary.csv', index=False)

print("✓ Saved: rfm_segmented.csv")
print("✓ Saved: rfm_segment_summary.csv")
print("✓ Saved: cluster_summary.csv")

### 8.5 Segmentation Visualizations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('CUSTOMER SEGMENTATION ANALYSIS', fontsize=18, fontweight='bold')

# 1. RFM Segment Distribution
segment_counts = rfm_df['RFM_Segment'].value_counts()
colors = plt.cm.Set3(range(len(segment_counts)))
segment_counts.plot(kind='barh', ax=axes[0, 0], color=colors)
axes[0, 0].set_title('Customer Distribution by RFM Segment', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Number of Customers')

# 2. Revenue by RFM Segment
segment_revenue = rfm_df.groupby('RFM_Segment')['Monetary'].sum().sort_values(ascending=False)
segment_revenue_billions = segment_revenue / 1e9
segment_revenue_billions.plot(kind='barh', ax=axes[0, 1], color=colors)
axes[0, 1].set_title('Total Revenue by RFM Segment (₦ Billions)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Revenue (₦B)')

# 3. Cluster Distribution
cluster_counts = rfm_df['Cluster_Name'].value_counts()
colors2 = plt.cm.Pastel1(range(len(cluster_counts)))
cluster_counts.plot(kind='barh', ax=axes[1, 0], color=colors2)
axes[1, 0].set_title('Customer Distribution by K-Means Cluster', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Number of Customers')

# 4. Revenue by Cluster
cluster_revenue = rfm_df.groupby('Cluster_Name')['Monetary'].sum().sort_values(ascending=False)
cluster_revenue_billions = cluster_revenue / 1e9
cluster_revenue_billions.plot(kind='barh', ax=axes[1, 1], color=colors2)
axes[1, 1].set_title('Total Revenue by Cluster (₦ Billions)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Revenue (₦B)')

plt.tight_layout()
plt.savefig('rfm_segmentation.png', dpi=300, bbox_inches='tight')
print("✓ Saved: rfm_segmentation.png")
plt.show()

## 🔮 PART 9: Predictive Modeling

### 9.1 Churn Prediction Model

In [ ]:
print("="*80)
print("CHURN PREDICTION MODEL")
print("="*80)

# Define churn
CHURN_THRESHOLD = 90
rfm_df['Is_Churned'] = (rfm_df['Recency'] > CHURN_THRESHOLD).astype(int)

print(f"\n✓ Defined churn threshold: {CHURN_THRESHOLD} days")
print(f"  Churned customers: {rfm_df['Is_Churned'].sum():,} ({rfm_df['Is_Churned'].mean()*100:.1f}%)")
print(f"  Active customers: {(1-rfm_df['Is_Churned']).sum():,} ({(1-rfm_df['Is_Churned'].mean())*100:.1f}%)")

# Prepare features
churn_features = ['Frequency', 'Monetary', 'Avg_Order_Value', 'Customer_Age_Days',
                  'Purchase_Rate', 'Recency', 'Total_Items_Sold']

category_cols = [col for col in rfm_df.columns if col.startswith('Category_')]
churn_features.extend(category_cols)

if 'R_Score' in rfm_df.columns:
    churn_features.extend(['R_Score', 'F_Score', 'M_Score'])

if 'Customer_Type' in rfm_df.columns:
    le_customer = LabelEncoder()
    rfm_df['Customer_Type_Encoded'] = le_customer.fit_transform(rfm_df['Customer_Type'])
    churn_features.append('Customer_Type_Encoded')

X_churn = rfm_df[churn_features].fillna(0)
y_churn = rfm_df['Is_Churned']

# Train-test split
X_train_churn, X_test_churn, y_train_churn, y_test_churn = train_test_split(
    X_churn, y_churn, test_size=0.2, random_state=42, stratify=y_churn
)

print(f"\n✓ Training set: {len(X_train_churn):,} samples")
print(f"✓ Test set: {len(X_test_churn):,} samples")

# Train model
print("\n🤖 Training Gradient Boosting Classifier...")
churn_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    subsample=0.8
)

churn_model.fit(X_train_churn, y_train_churn)

# Make predictions
y_pred_churn = churn_model.predict(X_test_churn)
y_pred_proba_churn = churn_model.predict_proba(X_test_churn)[:, 1]

# Evaluate
accuracy = accuracy_score(y_test_churn, y_pred_churn)
auc_roc = roc_auc_score(y_test_churn, y_pred_proba_churn)

print(f"\n✓ Churn Model Performance:")
print(f"  Accuracy: {accuracy*100:.1f}%")
print(f"  AUC-ROC: {auc_roc:.3f}")

# Predict for all customers
rfm_df['Churn_Probability'] = churn_model.predict_proba(X_churn)[:, 1]

# Categorize churn risk
def categorize_churn_risk(prob):
    if prob < 0.3:
        return 'Low'
    elif prob < 0.5:
        return 'Medium'
    elif prob < 0.7:
        return 'High'
    else:
        return 'Critical'

rfm_df['Churn_Risk_Level'] = rfm_df['Churn_Probability'].apply(categorize_churn_risk)

print(f"\n✓ Churn Risk Distribution:")
print(rfm_df['Churn_Risk_Level'].value_counts())

# Save model
with open('churn_model.pkl', 'wb') as f:
    pickle.dump(churn_model, f)
print("\n✓ Saved churn_model.pkl")

### 9.2 Customer Lifetime Value (CLV) Prediction

In [ ]:
print("\n" + "="*80)
print("CLV PREDICTION MODEL")
print("="*80)

# Prepare features
clv_features = ['Frequency', 'Avg_Order_Value', 'Customer_Age_Days',
                'Purchase_Rate', 'Recency', 'Total_Items_Sold']

clv_features.extend(category_cols)

if 'R_Score' in rfm_df.columns:
    clv_features.extend(['R_Score', 'F_Score', 'M_Score'])

if 'Customer_Type_Encoded' in rfm_df.columns:
    clv_features.append('Customer_Type_Encoded')

clv_data = rfm_df[rfm_df['Monetary'] > 0].copy()

X_clv = clv_data[clv_features].fillna(0)
y_clv = clv_data['Monetary']

# Train-test split
X_train_clv, X_test_clv, y_train_clv, y_test_clv = train_test_split(
    X_clv, y_clv, test_size=0.2, random_state=42
)

print(f"\n✓ Training set: {len(X_train_clv):,} samples")
print(f"✓ Test set: {len(X_test_clv):,} samples")

# Train model
print("\n🤖 Training Gradient Boosting Regressor...")
clv_model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    subsample=0.8
)

clv_model.fit(X_train_clv, y_train_clv)

# Make predictions
y_pred_clv = clv_model.predict(X_test_clv)

# Evaluate
r2 = r2_score(y_test_clv, y_pred_clv)
mae = mean_absolute_error(y_test_clv, y_pred_clv)
rmse = np.sqrt(mean_squared_error(y_test_clv, y_pred_clv))

print(f"\n✓ CLV Model Performance:")
print(f"  R² Score: {r2:.3f} ({r2*100:.1f}% variance explained)")
print(f"  MAE: ₦{mae:,.0f}")
print(f"  RMSE: ₦{rmse:,.0f}")

# Predict for all customers
X_all_clv = rfm_df[clv_features].fillna(0)
rfm_df['Predicted_CLV'] = clv_model.predict(X_all_clv)
rfm_df['Predicted_CLV'] = rfm_df['Predicted_CLV'].clip(lower=0)

# Categorize CLV
clv_percentiles = rfm_df['Predicted_CLV'].quantile([0.25, 0.50, 0.75, 0.90])

def categorize_clv(clv):
    if clv >= clv_percentiles[0.90]:
        return 'Very High Value'
    elif clv >= clv_percentiles[0.75]:
        return 'High Value'
    elif clv >= clv_percentiles[0.50]:
        return 'Medium Value'
    elif clv >= clv_percentiles[0.25]:
        return 'Low Value'
    else:
        return 'Very Low Value'

rfm_df['CLV_Category'] = rfm_df['Predicted_CLV'].apply(categorize_clv)

print(f"\n✓ CLV Distribution:")
print(rfm_df['CLV_Category'].value_counts())
print(f"\n  Total Predicted CLV: ₦{rfm_df['Predicted_CLV'].sum()/1e9:.2f}B")
print(f"  Average CLV: ₦{rfm_df['Predicted_CLV'].mean()/1e6:.2f}M")

# Save model
with open('clv_model.pkl', 'wb') as f:
    pickle.dump(clv_model, f)
print("\n✓ Saved clv_model.pkl")

### 9.3 Purchase Timing Analysis

In [ ]:
print("\n" + "="*80)
print("PURCHASE TIMING ANALYSIS")
print("="*80)

# Calculate expected days to next purchase
rfm_df['Days_Between_Purchases'] = rfm_df['Customer_Age_Days'] / rfm_df['Frequency'].replace(0, 1)
rfm_df['Expected_Days_to_Next_Purchase'] = rfm_df['Days_Between_Purchases']

rfm_df['Days_Since_Last_Purchase'] = rfm_df['Recency']
rfm_df['Days_Overdue'] = (rfm_df['Days_Since_Last_Purchase'] -
                          rfm_df['Expected_Days_to_Next_Purchase']).clip(lower=0)

# Categorize purchase timing
def categorize_purchase_timing(row):
    days_since = row['Days_Since_Last_Purchase']
    expected_days = row['Expected_Days_to_Next_Purchase']
    
    if row['Frequency'] == 1:
        return 'New/One-time'
    elif days_since < expected_days * 0.8:
        return 'Due Soon'
    elif days_since < expected_days * 1.2:
        return 'On Track'
    elif days_since < expected_days * 2:
        return 'Slightly Overdue'
    elif days_since < expected_days * 3:
        return 'Overdue'
    else:
        return 'Severely Overdue'

rfm_df['Purchase_Timing_Status'] = rfm_df.apply(categorize_purchase_timing, axis=1)

print(f"\n✓ Purchase Timing Distribution:")
print(rfm_df['Purchase_Timing_Status'].value_counts())

### 9.4 Customer Priority Scoring

In [ ]:
print("\n" + "="*80)
print("CUSTOMER PRIORITY SCORING")
print("="*80)

# Calculate customer value score
rfm_df['Customer_Value_Score'] = (
    (rfm_df['Predicted_CLV'] / rfm_df['Predicted_CLV'].max() * 50) +
    (rfm_df['Churn_Probability'] * 50)
)

# Categorize priority
def categorize_priority(row):
    clv = row['Predicted_CLV']
    churn_prob = row['Churn_Probability']
    
    high_clv = clv > rfm_df['Predicted_CLV'].quantile(0.75)
    high_churn = churn_prob > 0.5
    
    if high_clv and high_churn:
        return 'CRITICAL'
    elif high_clv:
        return 'High'
    elif high_churn and clv > rfm_df['Predicted_CLV'].quantile(0.5):
        return 'Medium'
    else:
        return 'Low'

rfm_df['Customer_Priority'] = rfm_df.apply(categorize_priority, axis=1)

print(f"\n✓ Customer Priority Distribution:")
print(rfm_df['Customer_Priority'].value_counts())

### 9.5 Save Prediction Results

In [ ]:
# Save complete dataset with predictions
rfm_df.to_csv('rfm_with_predictions.csv', index=False)
print("✓ Saved: rfm_with_predictions.csv")

# Save high-risk customers
high_risk = rfm_df[
    (rfm_df['Churn_Risk_Level'].isin(['High', 'Critical'])) &
    (rfm_df['Monetary'] > rfm_df['Monetary'].quantile(0.5))
].sort_values('Monetary', ascending=False)

high_risk[['Customer_ID', 'RFM_Segment', 'Monetary', 'Predicted_CLV',
           'Churn_Probability', 'Churn_Risk_Level', 'Customer_Priority']].to_csv(
    'high_risk_customers.csv', index=False
)
print(f"✓ Saved: high_risk_customers.csv ({len(high_risk):,} customers)")

# Save high-value opportunities
high_value = rfm_df[
    rfm_df['Predicted_CLV'] > rfm_df['Predicted_CLV'].quantile(0.90)
].sort_values('Predicted_CLV', ascending=False)

high_value[['Customer_ID', 'RFM_Segment', 'Monetary', 'Predicted_CLV',
            'Churn_Probability', 'CLV_Category']].to_csv(
    'high_value_opportunities.csv', index=False
)
print(f"✓ Saved: high_value_opportunities.csv ({len(high_value):,} customers)")

# Save action priority list
action_priority = rfm_df.sort_values('Customer_Value_Score', ascending=False)
action_priority[['Customer_ID', 'RFM_Segment', 'Monetary', 'Predicted_CLV',
                 'Churn_Probability', 'Purchase_Timing_Status', 
                 'Customer_Priority', 'Customer_Value_Score']].to_csv(
    'action_priority_list.csv', index=False
)
print(f"✓ Saved: action_priority_list.csv")

# Save model summary
model_summary = pd.DataFrame({
    'Model': ['Churn Prediction', 'CLV Prediction'],
    'Algorithm': ['Gradient Boosting Classifier', 'Gradient Boosting Regressor'],
    'Performance': [f'{auc_roc:.3f} (AUC-ROC)', f'{r2:.3f} (R²)'],
    'Training_Samples': [len(X_train_churn), len(X_train_clv)],
    'Features_Used': [len(churn_features), len(clv_features)]
})
model_summary.to_csv('model_summary.csv', index=False)
print(f"✓ Saved: model_summary.csv")

print("\n✅ All prediction models complete and saved!")

### 9.6 Prediction Visualizations

In [ ]:
# Churn Analysis Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('CHURN & CLV PREDICTION ANALYSIS', fontsize=18, fontweight='bold')

# 1. Churn Risk Distribution
risk_counts = rfm_df['Churn_Risk_Level'].value_counts()
risk_order = ['Low', 'Medium', 'High', 'Critical']
risk_counts = risk_counts.reindex(risk_order, fill_value=0)
risk_colors = {'Low': '#28a745', 'Medium': '#ffc107', 'High': '#fd7e14', 'Critical': '#dc3545'}
colors = [risk_colors[x] for x in risk_order]
axes[0, 0].bar(risk_order, risk_counts.values, color=colors)
axes[0, 0].set_title('Churn Risk Level Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Number of Customers')

# 2. ROC Curve
fpr, tpr, _ = roc_curve(y_test_churn, y_pred_proba_churn)
axes[0, 1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_roc:.3f})')
axes[0, 1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[0, 1].set_xlabel('False Positive Rate')
axes[0, 1].set_ylabel('True Positive Rate')
axes[0, 1].set_title('ROC Curve - Churn Prediction', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. CLV Distribution
axes[1, 0].hist(rfm_df['Predicted_CLV'], bins=50, color='green', edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Predicted CLV (₦)')
axes[1, 0].set_ylabel('Number of Customers')
axes[1, 0].set_title('Predicted CLV Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xscale('log')

# 4. CLV vs Churn Matrix
scatter = axes[1, 1].scatter(rfm_df['Churn_Probability'], rfm_df['Predicted_CLV'],
                             c=rfm_df['Customer_Value_Score'], cmap='RdYlGn_r',
                             alpha=0.6, s=30)
axes[1, 1].set_xlabel('Churn Probability')
axes[1, 1].set_ylabel('Predicted CLV (₦)')
axes[1, 1].set_title('CLV vs Churn Risk Matrix', fontsize=14, fontweight='bold')
axes[1, 1].set_yscale('log')
axes[1, 1].axvline(0.5, color='red', linestyle='--', linewidth=1, alpha=0.5)
plt.colorbar(scatter, ax=axes[1, 1], label='Priority Score')

plt.tight_layout()
plt.savefig('churn_prediction_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: churn_prediction_analysis.png")
plt.savefig('clv_prediction_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: clv_prediction_analysis.png")
plt.show()

In [ ]:
# Customer Priority Matrix
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('CUSTOMER PRIORITY MATRIX', fontsize=18, fontweight='bold')

# Priority Distribution
priority_counts = rfm_df['Customer_Priority'].value_counts()
colors_dict = {'CRITICAL': '#dc3545', 'High': '#fd7e14', 'Medium': '#ffc107', 'Low': '#28a745'}
priority_colors = [colors_dict.get(p, '#6c757d') for p in priority_counts.index]
priority_counts.plot(kind='bar', ax=axes[0], color=priority_colors)
axes[0].set_title('Customer Priority Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Customers')
axes[0].tick_params(axis='x', rotation=45)

# Priority by Revenue
priority_revenue = rfm_df.groupby('Customer_Priority')['Monetary'].sum() / 1e9
priority_revenue.plot(kind='bar', ax=axes[1], color=priority_colors)
axes[1].set_title('Total Revenue by Priority (₦ Billions)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Revenue (₦B)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('customer_priority_matrix.png', dpi=300, bbox_inches='tight')
print("✓ Saved: customer_priority_matrix.png")
plt.show()

## 🎯 PART 10: Product Recommendations

### 10.1 Prepare Product Data

In [ ]:
print("="*80)
print("PRODUCT RECOMMENDATION ENGINE")
print("="*80)

# Get category columns
category_cols = [col for col in rfm_df.columns if col.startswith('Category_')]
print(f"\n✓ Found {len(category_cols)} product categories")

# Create customer-product matrix
customer_product_matrix = rfm_df[['Customer_ID'] + category_cols].copy()
print(f"✓ Created customer-product matrix: {customer_product_matrix.shape}")

### 10.2 Collaborative Filtering

In [ ]:
def get_similar_customers(customer_id, customer_product_matrix, top_n=10):
    """Find similar customers based on purchase patterns"""
    target_purchases = customer_product_matrix[customer_product_matrix['Customer_ID'] == customer_id].iloc[0, 1:].values
    
    similarities = []
    for idx, row in customer_product_matrix.iterrows():
        if row['Customer_ID'] == customer_id:
            continue
        
        other_purchases = row[1:].values
        intersection = np.sum(np.minimum(target_purchases, other_purchases))
        union = np.sum(np.maximum(target_purchases, other_purchases))
        
        if union > 0:
            similarity = intersection / union
            similarities.append((row['Customer_ID'], similarity))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

def recommend_products_collaborative(customer_id, customer_product_matrix, top_n=5):
    """Recommend products based on similar customers"""
    similar_customers = get_similar_customers(customer_id, customer_product_matrix, top_n=20)
    
    if not similar_customers:
        return []
    
    target_row = customer_product_matrix[customer_product_matrix['Customer_ID'] == customer_id]
    if target_row.empty:
        return []
    
    target_purchases = target_row.iloc[0, 1:].values
    
    recommendations = defaultdict(float)
    
    for similar_id, similarity in similar_customers:
        similar_row = customer_product_matrix[customer_product_matrix['Customer_ID'] == similar_id]
        if similar_row.empty:
            continue
        
        similar_purchases = similar_row.iloc[0, 1:].values
        
        for i, category in enumerate(category_cols):
            if similar_purchases[i] > 0 and target_purchases[i] == 0:
                recommendations[category] += similarity
    
    sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    
    result = []
    for category, score in sorted_recs[:top_n]:
        category_name = category.replace('Category_', '')
        result.append({
            'Category': category_name,
            'Score': score,
            'Method': 'Collaborative Filtering'
        })
    
    return result

print("✓ Collaborative filtering functions defined")

### 10.3 Association Rules (Market Basket Analysis)

In [ ]:
def calculate_association_rules(customer_product_matrix, min_support=0.01, min_confidence=0.1):
    """Calculate product association rules"""
    purchase_matrix = customer_product_matrix.iloc[:, 1:].values > 0
    category_names = [col.replace('Category_', '') for col in category_cols]
    
    n_customers = len(purchase_matrix)
    
    item_support = {}
    for i, category in enumerate(category_names):
        support = np.sum(purchase_matrix[:, i]) / n_customers
        if support >= min_support:
            item_support[category] = support
    
    rules = []
    
    for i, cat1 in enumerate(category_names):
        if cat1 not in item_support:
            continue
        
        for j, cat2 in enumerate(category_names):
            if i >= j or cat2 not in item_support:
                continue
            
            both = np.sum(purchase_matrix[:, i] & purchase_matrix[:, j])
            support_both = both / n_customers
            
            if support_both < min_support:
                continue
            
            confidence_1_to_2 = support_both / item_support[cat1]
            confidence_2_to_1 = support_both / item_support[cat2]
            lift = support_both / (item_support[cat1] * item_support[cat2])
            
            if confidence_1_to_2 >= min_confidence:
                rules.append({
                    'Antecedent': cat1,
                    'Consequent': cat2,
                    'Support': support_both,
                    'Confidence': confidence_1_to_2,
                    'Lift': lift
                })
            
            if confidence_2_to_1 >= min_confidence:
                rules.append({
                    'Antecedent': cat2,
                    'Consequent': cat1,
                    'Support': support_both,
                    'Confidence': confidence_2_to_1,
                    'Lift': lift
                })
    
    return pd.DataFrame(rules)

# Calculate association rules
print("\nCalculating association rules...")
association_rules = calculate_association_rules(customer_product_matrix, min_support=0.01, min_confidence=0.1)
association_rules = association_rules.sort_values('Lift', ascending=False)

print(f"✓ Found {len(association_rules)} association rules")

if len(association_rules) > 0:
    print(f"\n📊 Top 5 Association Rules:")
    display(association_rules.head()[['Antecedent', 'Consequent', 'Confidence', 'Lift']])

# Save
association_rules.to_csv('product_association_rules.csv', index=False)
print(f"\n✓ Saved: product_association_rules.csv")

### 10.4 Hybrid Recommendation System

In [ ]:
def recommend_products_association(customer_id, customer_product_matrix, association_rules, top_n=5):
    """Recommend products based on association rules"""
    target_row = customer_product_matrix[customer_product_matrix['Customer_ID'] == customer_id]
    if target_row.empty:
        return []
    
    target_purchases = target_row.iloc[0, 1:].values
    purchased_categories = [col.replace('Category_', '') for i, col in enumerate(category_cols) if target_purchases[i] > 0]
    
    if not purchased_categories:
        return []
    
    recommendations = defaultdict(float)
    
    for category in purchased_categories:
        relevant_rules = association_rules[association_rules['Antecedent'] == category]
        
        for _, rule in relevant_rules.iterrows():
            consequent = rule['Consequent']
            consequent_col = f'Category_{consequent}'
            if consequent_col in category_cols:
                idx = category_cols.index(consequent_col)
                if target_purchases[idx] == 0:
                    score = rule['Confidence'] * rule['Lift']
                    recommendations[consequent] = max(recommendations[consequent], score)
    
    sorted_recs = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    
    result = []
    for category, score in sorted_recs[:top_n]:
        result.append({
            'Category': category,
            'Score': score,
            'Method': 'Association Rules'
        })
    
    return result

def get_hybrid_recommendations(customer_id, customer_product_matrix, association_rules, top_n=5):
    """Combine collaborative filtering and association rules"""
    collab_recs = recommend_products_collaborative(customer_id, customer_product_matrix, top_n=10)
    assoc_recs = recommend_products_association(customer_id, customer_product_matrix, association_rules, top_n=10)
    
    combined_scores = defaultdict(lambda: {'score': 0, 'methods': []})
    
    for rec in collab_recs:
        category = rec['Category']
        combined_scores[category]['score'] += rec['Score'] * 0.3
        combined_scores[category]['methods'].append('Collaborative')
    
    for rec in assoc_recs:
        category = rec['Category']
        combined_scores[category]['score'] += rec['Score'] * 0.7
        combined_scores[category]['methods'].append('Association')
    
    sorted_recs = sorted(combined_scores.items(), key=lambda x: x[1]['score'], reverse=True)
    
    recommendations = []
    for category, data in sorted_recs[:top_n]:
        normalized_score = min(1.0, data['score'] / 5.0)
        
        recommendations.append({
            'Customer_ID': customer_id,
            'Recommended_Category': category,
            'Confidence': normalized_score,
            'Reason': ', '.join(data['methods'])
        })
    
    return recommendations

print("✓ Hybrid recommendation system defined")

### 10.5 Generate Recommendations

In [ ]:
print("\nGenerating recommendations for top customers...")

# Generate for top 1000 customers
target_customers = rfm_df.nlargest(1000, 'Monetary')['Customer_ID'].tolist()
print(f"✓ Generating recommendations for {len(target_customers):,} customers...")

all_recommendations = []
for i, customer_id in enumerate(target_customers):
    if (i + 1) % 100 == 0:
        print(f"  Progress: {i+1}/{len(target_customers)} customers processed...")
    
    recs = get_hybrid_recommendations(customer_id, customer_product_matrix, association_rules, top_n=5)
    all_recommendations.extend(recs)

recommendations_df = pd.DataFrame(all_recommendations)
print(f"\n✓ Generated {len(recommendations_df):,} recommendations")

# Save recommendations
recommendations_df.to_csv('product_recommendations.csv', index=False)
print(f"✓ Saved: product_recommendations.csv")

# Top recommendations per customer
top_recs_per_customer = recommendations_df.sort_values(['Customer_ID', 'Confidence'], ascending=[True, False])
top_3_per_customer = top_recs_per_customer.groupby('Customer_ID').head(3)
top_3_per_customer.to_csv('top_recommendations_per_customer.csv', index=False)
print(f"✓ Saved: top_recommendations_per_customer.csv")

### 10.6 Cross-Sell Opportunities

In [ ]:
print("\nIdentifying cross-sell opportunities...")

cross_sell_opportunities = []

for _, customer in rfm_df.iterrows():
    customer_id = customer['Customer_ID']
    
    purchased_categories = sum([customer[col] for col in category_cols if col in customer.index])
    
    if purchased_categories < 3 and customer['Predicted_CLV'] > rfm_df['Predicted_CLV'].median():
        customer_recs = recommendations_df[recommendations_df['Customer_ID'] == customer_id]
        
        if len(customer_recs) > 0:
            top_rec = customer_recs.iloc[0]
            
            cross_sell_opportunities.append({
                'Customer_ID': customer_id,
                'RFM_Segment': customer.get('RFM_Segment', 'Unknown'),
                'Current_Categories': purchased_categories,
                'Predicted_CLV': customer['Predicted_CLV'],
                'Recommended_Category': top_rec['Recommended_Category'],
                'Confidence': top_rec['Confidence'],
                'Potential_Value': customer['Predicted_CLV'] * 0.2
            })

cross_sell_df = pd.DataFrame(cross_sell_opportunities)
cross_sell_df = cross_sell_df.sort_values('Potential_Value', ascending=False)

print(f"✓ Identified {len(cross_sell_df):,} cross-sell opportunities")
print(f"  Potential Revenue: ₦{cross_sell_df['Potential_Value'].sum()/1e9:.2f}B")

cross_sell_df.to_csv('cross_sell_opportunities.csv', index=False)
print(f"✓ Saved: cross_sell_opportunities.csv")

### 10.7 Recommendation Visualizations

In [ ]:
if len(recommendations_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('PRODUCT RECOMMENDATION ANALYSIS', fontsize=18, fontweight='bold')
    
    # 1. Top Recommended Categories
    top_categories = recommendations_df['Recommended_Category'].value_counts().head(10)
    colors = plt.cm.Set3(range(len(top_categories)))
    top_categories.plot(kind='barh', ax=axes[0, 0], color=colors)
    axes[0, 0].set_title('Top 10 Recommended Categories', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Number of Recommendations')
    
    # 2. Recommendation Method Distribution
    method_dist = recommendations_df['Reason'].value_counts()
    axes[0, 1].pie(method_dist.values, labels=method_dist.index, autopct='%1.1f%%', startangle=90)
    axes[0, 1].set_title('Recommendation Method Distribution', fontsize=14, fontweight='bold')
    
    # 3. Confidence Distribution
    axes[1, 0].hist(recommendations_df['Confidence'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(0.7, color='red', linestyle='--', linewidth=2, label='High Confidence (0.7)')
    axes[1, 0].set_xlabel('Confidence Score')
    axes[1, 0].set_ylabel('Number of Recommendations')
    axes[1, 0].set_title('Recommendation Confidence Distribution', fontsize=14, fontweight='bold')
    axes[1, 0].legend()
    
    # 4. Cross-sell Potential
    if len(cross_sell_df) > 0:
        rec_categories = cross_sell_df['Recommended_Category'].value_counts().head(10)
        colors2 = plt.cm.Paired(range(len(rec_categories)))
        rec_categories.plot(kind='barh', ax=axes[1, 1], color=colors2)
        axes[1, 1].set_title('Top Categories for Cross-sell', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Number of Opportunities')
    
    plt.tight_layout()
    plt.savefig('recommendation_analysis.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: recommendation_analysis.png")
    plt.savefig('cross_sell_opportunities.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: cross_sell_opportunities.png")
    plt.show()
else:
    print("No recommendations generated to visualize")

## 📈 PART 11: Final Summary

In [ ]:
print("="*80)
print("FINAL SUMMARY - AFRIMASH CUSTOMER INTELLIGENCE SOLUTION")
print("="*80)

print("\n📊 DATA PROCESSING:")
print(f"  Total Customers Analyzed: {len(rfm_df):,}")
print(f"  Total Transactions: {len(trans_df):,}")
print(f"  Total Revenue: ₦{trans_df['Revenue'].sum()/1e9:.2f}B")
print(f"  Features Created: {rfm_df.shape[1]}")

print("\n👥 CUSTOMER SEGMENTATION:")
print(f"  RFM Segments: {rfm_df['RFM_Segment'].nunique()}")
print(f"  K-Means Clusters: {rfm_df['Cluster_Name'].nunique()}")
print(f"  Top Segment: {rfm_df['RFM_Segment'].mode()[0]}")

print("\n🔮 PREDICTIVE MODELS:")
print(f"  Churn Model Accuracy: {accuracy*100:.1f}%")
print(f"  Churn Model AUC-ROC: {auc_roc:.3f}")
print(f"  CLV Model R² Score: {r2:.3f}")
print(f"  High-Risk Customers: {len(high_risk):,}")
print(f"  Revenue at Risk: ₦{high_risk['Monetary'].sum()/1e9:.2f}B")

print("\n🎯 RECOMMENDATIONS:")
if len(recommendations_df) > 0:
    print(f"  Total Recommendations: {len(recommendations_df):,}")
    print(f"  Customers Covered: {recommendations_df['Customer_ID'].nunique():,}")
    print(f"  Avg Confidence: {recommendations_df['Confidence'].mean():.2f}")
    print(f"  Cross-sell Opportunities: {len(cross_sell_df):,}")
    print(f"  Cross-sell Potential: ₦{cross_sell_df['Potential_Value'].sum()/1e9:.2f}B")
else:
    print("  No recommendations generated")

print("\n📁 FILES GENERATED:")
generated_files = [
    'rfm_clean.csv',
    'transactions_clean.csv',
    'rfm_segmented.csv',
    'rfm_with_predictions.csv',
    'high_risk_customers.csv',
    'high_value_opportunities.csv',
    'action_priority_list.csv',
    'product_recommendations.csv',
    'product_association_rules.csv',
    'cross_sell_opportunities.csv',
    'rfm_segment_summary.csv',
    'cluster_summary.csv',
    'model_summary.csv',
    'summary_statistics.csv',
    'churn_model.pkl',
    'clv_model.pkl'
]

for i, file in enumerate(generated_files, 1):
    print(f"  {i}. {file}")

print("\n📊 VISUALIZATIONS CREATED:")
viz_files = [
    'eda_dashboard.png',
    'deep_dive_analysis.png',
    'rfm_segmentation.png',
    'churn_prediction_analysis.png',
    'clv_prediction_analysis.png',
    'customer_priority_matrix.png',
    'recommendation_analysis.png',
    'cross_sell_opportunities.png'
]

for i, file in enumerate(viz_files, 1):
    print(f"  {i}. {file}")

print("\n" + "="*80)
print("✅ COMPLETE! ALL ANALYSIS AND FILES GENERATED SUCCESSFULLY")
print("="*80)

print("\n🎯 NEXT STEPS:")
print("  1. Review all generated CSV files for insights")
print("  2. Run the dashboard: streamlit run afrimash_dashboard.py")
print("  3. Check high_risk_customers.csv for immediate actions")
print("  4. Implement recommendations from action_priority_list.csv")
print("\n🌾 Afrimash Customer Intelligence Solution Ready! 🚀")